<a href="https://colab.research.google.com/github/jx-dohwan/Emotion_recognition_in_conversation.ipynb/blob/main/Emotion_recognition_in_conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install transformers==4.4.0
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import torch
torch.__version__

'1.13.1+cu116'

In [8]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



##  데이터 다운로드

In [9]:
!git clone https://github.com/declare-lab/MELD.git

Cloning into 'MELD'...
remote: Enumerating objects: 487, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 487 (delta 6), reused 0 (delta 0), pack-reused 475
Receiving objects: 100% (487/487), 8.12 MiB | 41.15 MiB/s, done.
Resolving deltas: 100% (254/254), done.


In [11]:
import glob # 어떤 리스트가 있는지 도출하는 것
data_path = "./MELD/data/MELD/*csv"
data_path_list = glob.glob(data_path)
print(data_path_list)

['./MELD/data/MELD/test_sent_emo.csv', './MELD/data/MELD/dev_sent_emo.csv', './MELD/data/MELD/train_sent_emo.csv']


## 데이터 확인

In [13]:
!head -5 './MELD/data/MELD/dev_sent_emo.csv' # head는 해당 파일의 앞부분을 보겠다.

Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
1,"Oh my God, he’s lost it. He’s totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
3,"Or! Or, we could go to the bank, close our accounts and cut them off at the source.",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
4,You’re a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"


In [14]:
# 데이터 출력
import csv #csv 확장자를 읽어들임임
for data_path in data_path_list:
  f = open(data_path, 'r')
  rdr = csv.reader(f)

  for line in rdr:
    print(line)
    break

  f.close()
  break


['Sr No.', 'Utterance', 'Speaker', 'Emotion', 'Sentiment', 'Dialogue_ID', 'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime']


## 세션으로 데이터 분할하기

[발화1, 발화2, 발화3] --> 감정예측한다는 것은 발화3에 해당하는 감정을
여기서 발화1, 발화2에 해당하는 감정이 있고, 이를 학습데이터로 사용해야된다.
--> [발화1], [발화1, 발화2]

//
모델이 roberta: bidirectional
: attention이 양방향

[발화1, 발화2, 발화3]: 여기서 발화2에 해당하는 감정을 학습한다.
: 동시에 발화1에 해당하는 감정을 학습한다.

In [38]:
# 데이터 저장
# roberta : bidirectional 
# attention이 양방향
def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])    
    return final_data
    
for data_path in data_path_list:
    f = open(data_path, 'r')
    rdr = csv.reader(f)# 리스트 하나씩 데이터 불러읽어 들이기
    
    """ 세션 데이터 저장할 것"""
    session_dataset = []
    session = []
    speaker_set = []
    
    """ 실제 데이터 저장 방식 """
    pre_sess = 'start'
    for i, line in enumerate(rdr):
        if i == 0:
            """ 저장할 데이터들 index 확인 """
            header  = line
            utt_idx = header.index('Utterance')
            speaker_idx = header.index('Speaker')
            emo_idx = header.index('Emotion')
            sess_idx = header.index('Dialogue_ID')
        else:
            utt = line[utt_idx]
            speaker = line[speaker_idx]
            """ 유니크한 스피커로 바꾸기 """
            if speaker in speaker_set:# 스키퍼셋안에 있는 스피커(즉 이미 있는 스피커라면)
                uniq_speaker = speaker_set.index(speaker)# 어떤 사람이 말했는지 그 사람의 index를 uniq로 둔다.
            else:# 처음 등장한 인물이라면
                speaker_set.append(speaker)# 스피커셋에 스피커를 넣어주고
                uniq_speaker = speaker_set.index(speaker)#유니크 스피커가 몇 번째 인덱스에 해당하는지 바꿔줌
            emotion = line[emo_idx]
            sess = line[sess_idx]
            
            if pre_sess == 'start' or sess == pre_sess:# sess이 pre_sess이 같거나 start일때 저장
                session.append([uniq_speaker, utt, emotion])
            else:#다를경우 초기화하고 다시저장?
                """ 세션 데이터 저장 """
                # session_dataset.append(session)
                session_dataset += split(session)# 서브세셔으로 분리하여 데이터로 바꿈
                session = [[uniq_speaker, utt, emotion]]
                speaker_set = []
            pre_sess = sess   
    """ 마지막 세션 저장 """
    # session_dataset.append(session)
    session_dataset += split(session)
    f.close()
    
    """ 데이터 분할하기 """
    break

In [39]:
session_dataset[0]

[[0, 'Why do all you’re coffee mugs have numbers on the bottom?', 'surprise']]

In [40]:
session_dataset[1]

[[0, 'Why do all you’re coffee mugs have numbers on the bottom?', 'surprise'],
 [1,
  'Oh. That’s so Monica can keep track. That way if one on them is missing, she can be like, ‘Where’s number 27?!’',
  'anger']]